In [ ]:
# Query 1: How many tweets have location not null?

# Query 2: How many people have more followers than friends?

# Query 3: Return text of tweets which have the string "http://" ?

# Query 4: Return all the tweets which contain text "England" but not "UEFA" ?

# Query 5: Get all the tweets from the location "Ireland" and contains the string "UEFA"? 

In [ ]:
# Initialize mongodb on Ubuntu.
# Create database called "twitter"
use twitter
# create collection called "week6"
db.createCollection("week6")
# Import the "twitter.json" file to the mondoDB: 
# (the collection "week6" is going to inherit the content of the content of "twitter.json")
mongoimport --db twitter --collection week6 --file /home/tiagor/Desktop/coursera-master/big-data-3/twitter.json

In [ ]:
# simple querys:
db.week6.findOne()
db.week6.distinct("Location")
db.week6.distinct("user_name")

In [ ]:
_id, user_name, retweet_count, tweet_followers_count, source, coordinates, tweet_mentioned_count, tweet_ID, tweet_text, user.CreatedAt, user.FavouritesCount, user.FollowersCount, user.FriendsCount, user.UserId, user.Location

In [ ]:
# Query 1:
> db.week6.countDocuments({ "user.Location": { $ne: null } })
# Result:
6937

# Export query to a csv file: (db.week6.find({ "user.Location": { $ne: null } }))
mongoexport --db twitter --collection week6 --query '{ "user.Location": { "$ne": null } }' --type=csv --fields _user.Location --out /home/tiagor/Desktop/coursera-master/big-data-3/week_6/query_01.csv
mongoexport --uri "mongodb://localhost/twitter" --collection week6 --query '{ "user.Location": { "$ne": null } }' --type=csv --fields "_id,user_name,retweet_count,tweet_followers_count,source,coordinates,tweet_mentioned_count,tweet_ID,tweet_text,user.CreatedAt,user.FavouritesCount,user.FollowersCount,user.FriendsCount,user.UserId,user.Location" --out /home/tiagor/Desktop/coursera-master/big-data-3/week_6/query_1.csv

In [ ]:
# Query 2:
> db.week6.find({ $expr: { $gt: ["$tweet_followers_count", "$user.FriendsCount"] } }).count()
# Result:
5809

# Export query to a csv file:
mongoexport --uri "mongodb://localhost/twitter" --collection week6 --query '{ "$expr": { "$gt": ["$tweet_followers_count", "$user.FriendsCount"] } }' --type=csv --fields "_id,user_name,retweet_count,tweet_followers_count,source,coordinates,tweet_mentioned_count,tweet_ID,tweet_text,user.CreatedAt,user.FavouritesCount,user.FollowersCount,user.FriendsCount,user.UserId,user.Location" --out /home/tiagor/Desktop/coursera-master/big-data-3/week_6/query_2.csv


In [ ]:
# Query 3: 
# simpler answer: (return all the output of "tweet_text"):
> db.week6.find({ tweet_text: { $regex: /http:/i } }, { tweet_text: 1, _id: 0 })
# complex answer: (return the output of "tweet_text" until the first ":" appears):
db.week6.aggregate([
  {
    $match: {
      tweet_text: { $regex: /http:/i }
    }
  },
  {
    $project: {
      _id: 0,
      tweet_text: {
        $trim: {
          input: {
            $substr: [
              "$tweet_text",
              0,
              {
                $indexOfBytes: ["$tweet_text", ":"]
              }
            ]
          }
        }
      }
    }
  }
])


# Result:
{ "tweet_text" : "RT @BWGotheem" }
{ "tweet_text" : "RT @TerraceImages" }
{ "tweet_text" : "http" }
{ "tweet_text" : "RT @espn" }
{ "tweet_text" : "RT @espn" }
{ "tweet_text" : "RT @newfy320" }
{ "tweet_text" : "RT @izzy5115" }
{ "tweet_text" : "RT @Ass0Star" }
{ "tweet_text" : "RT @FootyFunnysUK" }
{ "tweet_text" : "RT @trevorphibbs" }

# Export query to a csv file:
mongoexport --db twitter --collection week6 --query '{ "tweet_text": { "$regex": "http://", "$options": "i" } }' --type=csv --fields "tweet_text" --out /home/tiagor/Desktop/coursera-master/big-data-3/week_6/query_03.csv
mongoexport --uri "mongodb://localhost/twitter" --collection week6 --query '{ "tweet_text": { "$regex": "http://", "$options": "i" } }' --type=csv --fields "_id,user_name,retweet_count,tweet_followers_count,source,coordinates,tweet_mentioned_count,tweet_ID,tweet_text,user.CreatedAt,user.FavouritesCount,user.FollowersCount,user.FriendsCount,user.UserId,user.Location" --out /home/tiagor/Desktop/coursera-master/big-data-3/week_6/query_3.csv


In [ ]:
# Query 4:
# simpler answer: (return all the output of "tweet_text"):
> db.week6.find({$and: [{ tweet_text: {$regex: /England/i}}, {tweet_text: {$not: /UEFA/i}} ] }, {tweet_text: 1, _id: 0})
# complex answer: (return the output of "tweet_text" that contins "Euro 2016" and count them):
db.week6.find(
  {
    $and: [
      { tweet_text: { $regex: /England/i } },
      { tweet_text: { $not: /UEFA/i } }
    ]
  },
  { tweet_text: 1, _id: 0 }
).forEach(function(doc) {
  var count = (doc.tweet_text.match(/Euro 2016/gi) || []).length;
  print("Occurrences of 'Euro 2016':", count);
})


# Result:
# In these results the string “Euro 2016” appears in: 2 tweests

# Export query to a csv file:
mongoexport --db twitter --collection week6 --query '{ "tweet_text": { "$regex": "England", "$options": "i", "$not": { "$regex": "UEFA", "$options": "i" } } }' --type=csv --fields tweet_text --out /home/tiagor/Desktop/coursera-master/big-data-3/week_6/query_04.csv
mongoexport --uri "mongodb://localhost/twitter" --collection week6 --query '{ "tweet_text": { "$regex": "England", "$options": "i", "$not": { "$regex": "UEFA", "$options": "i" } } }' --type=csv --fields "_id,user_name,retweet_count,tweet_followers_count,source,coordinates,tweet_mentioned_count,tweet_ID,tweet_text,user.CreatedAt,user.FavouritesCount,user.FollowersCount,user.FriendsCount,user.UserId,user.Location" --out /home/tiagor/Desktop/coursera-master/big-data-3/week_6/query_4.csv


In [ ]:
# Query 5:
> db.week6.find(
  {
    $and: [
      { "user.Location": "Ireland" },
      { tweet_text: { $regex: /UEFA/i } }
    ]
  },
)


# Result:
{ "_id" : ObjectId("578ffc547eb951401527b5da"), "user_name" : "ProfitwatchInfo", "retweet_count" : 0, "tweet_followers_count" : 602, "source" : "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>", "coordinates" : null, "tweet_mentioned_count" : 1, "tweet_ID" : "755893533047914498", "tweet_text" : "RT @DundalkFC: Full Time FH 2-2 Dundalk, Dundalk qualify to the next round of the UEFA Champions League!", "user" : { "CreatedAt" : ISODate("2016-03-31T19:06:24Z"), "FavouritesCount" : 1456, "FollowersCount" : 602, "FriendsCount" : 2381, "UserId" : -552161280, "Location" : "Ireland" } }
{ "_id" : ObjectId("578ffda17eb951401527b738"), "user_name" : "KillarneyKnight", "retweet_count" : 0, "tweet_followers_count" : 956, "source" : "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>", "coordinates" : null, "tweet_mentioned_count" : 1, "tweet_ID" : "755894929071702020", "tweet_text" : "RT @DundalkFC: Full Time FH 2-2 Dundalk, Dundalk qualify to the next round of the UEFA Champions League!", "user" : { "CreatedAt" : ISODate("2011-02-04T14:44:10Z"), "FavouritesCount" : 3015, "FollowersCount" : 956, "FriendsCount" : 2277, "UserId" : 247307380, "Location" : "Ireland" } }
{ "_id" : ObjectId("57965ef5c38159118f94f8ae"), "user_name" : "leedsfc9", "retweet_count" : 0, "tweet_followers_count" : 1380, "source" : "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>", "coordinates" : null, "tweet_mentioned_count" : 2, "tweet_ID" : "757648699917033477", "tweet_text" : "@brooughy26 @AstonVillaMedia Birmingham I guess once won the League Cup and little Fulham UEFA Cup final + Ipswich Historically Bobby Robson", "user" : { "CreatedAt" : ISODate("2014-05-27T16:54:33Z"), "FavouritesCount" : 14568, "FollowersCount" : 1380, "FriendsCount" : 802, "UserId" : -1727442750, "Location" : "Ireland" } }
{ "_id" : ObjectId("57966dc0c38159201ca7f637"), "user_name" : "Insight4News4", "retweet_count" : 0, "tweet_followers_count" : 358, "source" : "<a href=\"http://insight4news.ucd.ie\" rel=\"nofollow\">insight4news4</a>", "coordinates" : null, "tweet_mentioned_count" : 0, "tweet_ID" : "757664582127288321", "tweet_text" : "Liverpool fined by UEFA for Europa League final crowd trouble #liverpool https://t.co/vuC4Xesdl9", "user" : { "CreatedAt" : ISODate("2015-03-23T15:48:32Z"), "FavouritesCount" : 0, "FollowersCount" : 358, "FriendsCount" : 0, "UserId" : -1187124583, "Location" : "Ireland" } }
# the "user_name" = ProfitwatchInfo" is the one with the higher "FriendsCount".

# Export query to a csv file:
mongoexport --db twitter --collection week6 --query '{ "$and": [{ "user.Location": "Ireland" }, { "tweet_text": { "$regex": "UEFA", "$options": "i" } }] }' --type=csv --fields "tweet_text" --out /home/tiagor/Desktop/coursera-master/big-data-3/week_6/query_05.csv
mongoexport --uri "mongodb://localhost/twitter" --collection week6 --query '{ "$and": [{ "user.Location": "Ireland" }, { "tweet_text": { "$regex": "UEFA", "$options": "i" } }] }' --type=csv --fields "_id,user_name,retweet_count,tweet_followers_count,source,coordinates,tweet_mentioned_count,tweet_ID,tweet_text,user.CreatedAt,user.FavouritesCount,user.FollowersCount,user.FriendsCount,user.UserId,user.Location" --out /home/tiagor/Desktop/coursera-master/big-data-3/week_6/query_5.csv


In [ ]:
# Create the tweet_text.csv file necessary for the secound part of the week_6 project.

# Query to find all the tweet_text values:
> db.week6.find({}, { "tweet_text": 1, _id: 0})

# Export query to a csv file:
mongoexport --db twitter --collection week6 --query '{ "tweet_text": { "$exists": true } }' --type=csv --fields "tweet_text" --out /home/tiagor/PycharmProjects/Apache_Spark_Projects_and_Courses/Big_Data_Integration_and_Processing_exercises/week_6/tweet_text.csv
